In [9]:
import tensorflow as tf

def box_cxcywh_to_xyxy(boxes_raw):
    """ 
    Helper function used to convert raw box output to the 
    top left and bottom right point of the box.
    
    Params:
    - boxes_raw: Tensor of shape (N, 4) containing N boxes in (center x, center y, width, height) format.

    Returns:
    - box_coords: Tensor of shape (N, 4) containing N boxes in (x_topleft, y_topleft, x_bottomright, y_bottomright) format.
    """
    cx, cy, w, h = tf.unstack(boxes_raw, axis=1)

    x_topleft = cx - 0.5 * w
    y_topleft = cy - 0.5 * h
    x_bottomright = cx + 0.5 * w
    y_bottomright = cy + 0.5 * h
    
    box_coords = tf.stack([x_topleft, y_topleft, x_bottomright, y_bottomright], axis=1)
    
    return box_coords

def box_area(boxes):
    """ 
    Helper function used to calculate a bboxes' area given two corners.
    
    Params:
    - boxes: Tensor of shape (N, 4) containing N bounding boxes in (x_topleft, y_topleft, x_bottomright, y_bottomright) format.

    Returns:
    - areas: Tensor of shape (N,) containing the area of each bounding box.
    """
    xmin, ymin, xmax, ymax = tf.split(boxes, 4, axis=1)
    
    # Compute the width and height of each bounding box
    width = tf.squeeze(xmax - xmin, axis=1)
    height = tf.squeeze(ymax - ymin, axis=1)
    
    # Compute the area of each bounding box
    areas = width * height
    
    return areas


def box_iou(box_preds, box_trues):
    """
    Compute the intersection over union (IoU) between pairs of bounding boxes with matching indices.

    Args:
    - box_preds: Tensor of shape (N, 4) containing N predicted bounding boxes in (x_topleft, y_topleft, x_bottomright, y_bottomright) format.
    - box_trues: Tensor of shape (N, 4) containing N ground truth bounding boxes in (x_topleft, y_topleft, x_bottomright, y_bottomright) format.

    Returns:
    - iou: Tensor of shape (N,) containing the IoU between each pair of bounding boxes with matching indices.
    """
    # Compute areas of bounding boxes
    area_preds = box_area(box_preds)
    area_trues = box_area(box_trues)

    # Compute intersection coordinates
    xmin_inter = tf.maximum(box_preds[:, 0], box_trues[:, 0])
    ymin_inter = tf.maximum(box_preds[:, 1], box_trues[:, 1])
    xmax_inter = tf.minimum(box_preds[:, 2], box_trues[:, 2])
    ymax_inter = tf.minimum(box_preds[:, 3], box_trues[:, 3])

    # Compute intersection areas
    width_inter = tf.maximum(0.0, xmax_inter - xmin_inter)
    height_inter = tf.maximum(0.0, ymax_inter - ymin_inter)
    area_inter = width_inter * height_inter

    # Compute union areas
    area_union = area_preds + area_trues - area_inter

    # Compute IoU
    iou = area_inter / area_union

    return iou, area_union

In [10]:
def giou(box_preds, box_trues):
    """
    Calculate the Generalized IoU (GIoU) between two bounding boxes using TensorFlow.

    Args:
    - box_preds: Tensor of shape (N, 4) containing N bounding boxes in (x_topleft, y_topleft, x_bottomright, y_bottomright) format.
    - box_trues: Tensor of shape (N, 4) containing N bounding boxes in (x_topleft, y_topleft, x_bottomright, y_bottomright) format.

    Returns:
    - giou: Tensor of shape (N,) containing the Generalized IoU (GIoU) between the two bounding boxes.
    """
    # Compute intersection over union (IoU) and union using the existing function
    iou, union = box_iou(box_preds, box_trues)

    # Compute enclosing box (minimum area box that encloses both boxes)
    xmin_enclose = tf.minimum(box_preds[:, 0], box_trues[:, 0])
    ymin_enclose = tf.minimum(box_preds[:, 1], box_trues[:, 1])
    xmax_enclose = tf.maximum(box_preds[:, 2], box_trues[:, 2])
    ymax_enclose = tf.maximum(box_preds[:, 3], box_trues[:, 3])
    area_enclose = (xmax_enclose - xmin_enclose) * (ymax_enclose - ymin_enclose)

    # Compute Generalized IoU (GIoU)
    giou = iou - (area_enclose - union) / tf.maximum(area_enclose, 1e-10)

    return giou

def huber_loss(box_preds, box_trues):
    """
    Compute the Huber loss (smooth-L1) for bounding boxes.

    Parameters:
    - box_preds: Predicted bounding boxes of shape (N, 4) representing (xmin, ymin, xmax, ymax).
    - box_trues: Target bounding boxes of shape (N, 4) representing (xmin, ymin, xmax, ymax).

    Returns:
    - l1_loss: Scalar tensor representing the Huber error (smooth-L1) between the predicted and target bounding boxes.
    """
    loss_fn = tf.keras.losses.Huber()
    huber_loss = loss_fn(box_trues, box_preds)
    
    return huber_loss


In [ ]:
def calculate_loss(box_preds, box_trues):
    pass

In [11]:

def tests():
    box_preds = tf.constant([[1.0, 2.0, 5.0, 6.0], [2.0, 3.0, 6.0, 7.0]])
    box_trues = tf.constant([[3.0, 4.0, 7.0, 8.0], [4.0, 5.0, 8.0, 9.0]])
    
    iou_test = box_iou(box_preds, box_trues)
    print(iou_test[0].numpy()) # printing ious
    print(iou_test[1].numpy()) # printing unions (used for gIoU)
    assert(iou_test[0][0] == 1/7)
    assert(iou_test[0][1] == 1/7) 

    giou_test = giou(box_preds, box_trues)
    print(giou_test)
    assert(tf.round(giou_test[0], 5) == tf.round(-5/63, 5))

    huber_test = huber_loss(box_preds, box_trues)
    print(huber_test)
    assert(huber_test == 1.5)

tests()

[0.14285715 0.14285715]
[28. 28.]
tf.Tensor([-0.07936507 -0.07936507], shape=(2,), dtype=float32)
tf.Tensor(1.5, shape=(), dtype=float32)
